In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
def gaussian_filter(image_color, k_size=5):
    #gray scale
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    gaussian   = cv2.GaussianBlur(image_gray,(k_size,k_size), cv2.BORDER_DEFAULT)
    
    return gaussian

In [3]:
def median_filter(image_color, k_size=5): #good to deal with Salt & Pepper
    #gray scale
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    median     = cv2.medianBlur(image_gray, k_size)
    
    return median

In [4]:
def bilateral_filter(image_color,k_size=15,s_color=80,s_space=80): #Spatial and Range kernels
    #gray scale
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    bilateral  = cv2.bilateralFilter(image_gray,k_size,s_color,s_space)
    
    return bilateral

In [5]:
# def joint_bilateral_filter(image_color):

In [6]:
# def guided_image_filter(image_color):

In [7]:
def find_gradients_n_direction(x_sobel, y_sobel):
    #gradient
    G = np.hypot(x_sobel, y_sobel)
    G = G/ G.max() * 255
    #direction
    D = np.arctan2(x_sobel, y_sobel)
    
    return (G, D)

In [8]:
def non_maximum_suppression(image, D): #image, direction
    M, N = image.shape
    Z = np.zeros((M,N), dtype=np.int32)
    #angle
    A = D * 180. / np.pi
    A[A < 0] += 180
    
    #calcultation: avoid the edge 
    for i in range(1, M-1):
        for j in range(1, N-1):
            try: 
                q = 255
                r = 255
                
                #angle 0
                if (0 <= A[i,j] < 22.5) or (157.5 <= A[i,j] <= 180):
                    q = image[i, j+1]
                    r = image[i, j-1]
                #angle 45
                elif (22.5 <= A[i,j] < 67.5):
                    q = image[i+1, j-1]
                    r = image[i-1, j+1]
                #angle 90
                elif (67.5 <= A[i,j] < 112.5):
                    q = image[i+1, j]
                    r = image[i-1, j]
                #angle 135
                elif (112.5 <= A[i,j] < 157.5):
                    q = image[i-1, j-1]
                    r = image[i+1, j+1]
                    
                #non maximum suppression
                if(image[i,j]>=q) and (image[i,j]>=r):
                    Z[i,j] = image[i,j]
                else:
                    Z[i,j] = 0
            
            except IndexError as e: #???
                pass
    
    return Z.astype(np.uint8)

In [9]:
def double_threshold(image, low_TH_ratio=0.05, high_TH_ratio=0.09):
    
    high_TH = image.max() * high_TH_ratio
    low_TH  = high_TH * low_TH_ratio
    
    M, N = image.shape
    R = np.zeros((M, N), dtype=np.int32)
    
    #weak and strong edge pixel values
    weak   = np.int32(25)
    strong = np.int32(255)
    
    strong_i, strong_j = np.where(image >= high_TH)
    zeros_i, zeros_j = np.where(image < low_TH)
    weak_i, weak_j = np.where((image < high_TH) & (image >= low_TH))
    
    R[strong_i, strong_j] = strong
    R[weak_i, weak_j] = weak
    
    return(R, weak, strong)

In [10]:
def hysteresis(image, weak=25, strong=255):
    M, N = image.shape
    
    #avoid the edge
    for i in range(1, M-1):
        for j in range(1, N-1):
            if(image[i,j] == weak):
                try:
                    if((image[i+1,j-1]==strong)or(image[i+1,j]==strong)or(image[i+1,j+1]==strong)
                       or(image[i,j-1]==strong)or(image[i,j+1]==strong)
                      or(image[i-1,j-1]==strong)or(image[i-1,j]==strong)or(image[i-1,j+1]==strong)):
                        image[i, j] = strong
                    else:
                        image[i, j] = 0
                
                except IndexError as e: #??
                    pass
    
    return image.astype(np.uint8)
    
    

In [11]:
def canny(image_color):
    threshold_1 = 30
    threshold_2 = 80
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(image_gray, threshold_1, threshold_2)
    
    return canny



In [12]:
def laplace(image_color):
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(image_gray, cv2.CV_64F)
    
    return laplacian

In [13]:
def sobel_x(image_gray, kernel_size=7):
#     image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    x_sobel = cv2.Sobel(image_gray, cv2.CV_64F, 1, 0, ksize = kernel_size)
    
    return x_sobel

In [14]:
def sobel_y(image_gray, kernel_size=7):
#     image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    y_sobel = cv2.Sobel(image_gray, cv2.CV_64F, 0, 1, ksize = kernel_size)
    
    return y_sobel

In [15]:
def whole_process(img):
    x_sobel      = sobel_x(img)
    y_sobel      = sobel_y(img)
    
    #find the gradients 
    grads, directions = find_gradients_n_direction(x_sobel, y_sobel)

    #non-maximum suppression
    non_max_out  = non_maximum_suppression(grads, directions)

    #double threshold
    threshold_out, weak, strong= double_threshold(non_max_out)

    #hysteresis
    result       = hysteresis(threshold_out, weak, strong)
    
    return result

In [16]:
#Main Function

if __name__ == "__main__":
    
    me  = cv2.imread("canny.jpeg")
#     cv2.imshow('ME', me)
    
    Gaussian_ = gaussian_filter(me)

    x_sobel = sobel_x(Gaussian_)
    y_sobel = sobel_y(Gaussian_)

    Gradients_, Directions_ = find_gradients_n_direction(x_sobel, y_sobel)
    
    Non_max_sup = non_maximum_suppression(Gradients_, Directions_)
    
    cv2.imshow('Non-Maximum Suppression', Non_max_sup)
    
             
    cv2.waitKey()
    cv2.destroyAllWindows()
    
#     while True:
#         ret, frame = cap.read() # Cap.read() returns a ret bool to indicate success.
# #         cv2.imshow('Live Edge Detection', canny(frame))
# #         frame = cv2.imread('tmp.JPG')
        
#         gaussian_out = whole_process(gaussian_filter(frame))
#         median_out   = whole_process(median_filter(frame))
#         bilateral_out= whole_process(bilateral_filter(frame))
        
#         #show image
# #         cv2.imshow('Gaussian', gaussian_out)
# #         cv2.imshow('Median', median_out)
# #         cv2.imshow('Bilateral', bilateral_out)
#         cv2.imshow('Canny', canny(frame))
        
#         #real frame
# #         cv2.imshow('Webcam Video', frame)
#         if cv2.waitKey(1) == 13: #13 Enter Key
#             break

#     cap.release() # camera release 
#     cv2.destroyAllWindows()   
    